In [1]:
import numpy as np
import sys,os
sys.path.append('../LIB/')
from env import ENV
from tqdm import tqdm
import pandas as pd

# binary search

In [52]:
pv = 470790
length = 12
payment = 47041

def calculate_payment(pv,length,rate):
    return rate *pv / (1- (1+rate)**(-length))

def calculate_rate(pv,length,payment,space=np.arange(0,1,0.00001)):
    assert length * payment >= pv
    target = payment
    low = 0
    up = len(space)
    space_len = len(space)
    while up - low > 1:
        cur_index = int((low+up) / 2)

        rate = space[cur_index]
        cal = calculate_payment(pv,length,rate)
        if cal == target:
            return rate
        elif cal < target:
            low = cur_index
        else:
            up = cur_index
    if up == space_len:
        up -= 1
    up_cal = calculate_payment(pv,length,space[up])
    low_cal = calculate_payment(pv,length,space[low])
    if abs(up_cal - target) <= abs(low_cal - target):
        return space[up]
    else:
        return space[low]
        
        
def estimate_terms(pv,p,ns,t_apr=0.03,nu=60):
    low = ns
    up = nu
    while up - low > 1:
        cur_n = int((low+up) / 2)
        cal_apr = calculate_rate(pv,cur_n,p)
        if cal_apr == t_apr:
            return cal_apr,cur_n
        elif cal_apr < t_apr:
            low = cur_n
        elif cal_apr > t_apr:
            up = cur_n
    up_cal = calculate_rate(pv,up,p)
    low_cal = calculate_rate(pv,low,p)
    if abs(up_cal - t_apr) <= abs(low_cal - t_apr):
        return up_cal,up
    else:
        return low_cal,low

# Load files 

In [109]:
pre_ori = pd.read_csv(ENV.previous_application_ori.value)
print(pre_ori.shape)

# ins_ori = pd.read_csv(ENV.installments_payments_ori.value)
# print(ins_ori.shape)

# pos_ori = pd.read_csv(ENV.POS_CASH_balance_ori.value)
# print(pos_ori.shape)

# app_ori = pd.read_csv(ENV.application_train_ori.value)
# print(app_ori.shape)
# app_ori_test = pd.read_csv(ENV.application_test_ori.value)
# print(app_ori.shape)

# X = pd.concat([app_ori.drop('TARGET',axis=1),app_ori_test])
# print(X.shape)

(1670214, 37)


## Get Loan Type

In [4]:
X.NAME_CONTRACT_TYPE.value_counts()

Cash loans         326537
Revolving loans     29718
Name: NAME_CONTRACT_TYPE, dtype: int64

In [130]:
X_cash = X[X.NAME_CONTRACT_TYPE=='Cash loans'][['AMT_CREDIT','AMT_ANNUITY','SK_ID_CURR']].copy()

## View N/A

In [131]:
X_cash.isnull().sum()

AMT_CREDIT      0
AMT_ANNUITY    36
SK_ID_CURR      0
dtype: int64

In [132]:
print('train')
print(app_ori[app_ori.NAME_CONTRACT_TYPE=='Cash loans'].AMT_ANNUITY.isnull().sum())
print('test')
print(app_ori_test[app_ori_test.NAME_CONTRACT_TYPE=='Cash loans'].AMT_ANNUITY.isnull().sum())

train
12
test
24


In [133]:
print(X_cash.shape)
print('after drop na')
X_cash = X_cash.dropna()
print(X_cash.shape)

(326537, 3)
after drop na
(326501, 3)


# Step 1. add new col lowLimitLength

In [134]:
X_cash['lowLimitLength'] = np.ceil(X_cash.AMT_CREDIT.values/X_cash.AMT_ANNUITY.values)
X_cash['lowLimitLength'] = X_cash['lowLimitLength'].astype('int')

In [135]:
pv =135000
n = 12
payment = 16020
calculate_rate(pv,n,payment)

0.059070000000000004

In [136]:
pv =500490.0
n = 12
payment = 56731.5
calculate_rate(pv,n,payment)

0.05083000000000001

### for those have length 9.

In [29]:
X_cash_9 = X_cash[X_cash.lowLimitLength==9].copy()
pv_9 = X_cash_9.AMT_CREDIT.values
payment_9 = X_cash_9.AMT_ANNUITY.values
rate_9 = []
for index in range(len(X_cash_9)):
    pv = pv_9[index]
    p = payment_9[index]
    n = 12
    rate_9.append(calculate_rate(pv,n,p))
X_cash_9['apr'] = rate_9
    

### for those have length x, and approximate length q

In [ ]:
low_limit_length = [9,]
map2l = [12,]
counts = [7153,]
min_l = [0.04991,]
max_l = [0.06780,]

In [46]:
x = 9
q = 9

X_cash_x = X_cash[X_cash.lowLimitLength==x].copy()
pv_x = X_cash_x.AMT_CREDIT.values
payment_x = X_cash_x.AMT_ANNUITY.values
rate_x = []
for index in range(len(X_cash_x)):
    pv = pv_x[index]
    p = payment_x[index]
    n = q
    rate_x.append(calculate_rate(pv,n,p))
X_cash_x['apr'] = rate_x

In [60]:
pv =58500.0
n = 120
payment = 6597.0
calculate_rate(pv,n,payment)

0.11277000000000001

In [ ]:
def calculate_rate(pv,length,payment,space=np.arange(0,1,0.00001)):
    assert length * payment >= pv
    target = payment
    low = 0
    up = len(space)
    while up - low > 1:
        cur_index = int((low+up) / 2)

        rate = space[cur_index]
        cal = calculate_payment(pv,length,rate)
        if cal == target:
            return rate
        elif cal < target:
            low = cur_index
        else:
            up = cur_index
    up_cal = calculate_payment(pv,length,space[up])
    low_cal = calculate_payment(pv,length,space[low])
    if abs(up_cal - target) <= abs(low_cal - target):
        return space[up]
    else:
        return space[low]

In [164]:
def estimate_terms(pv,p,ns,t_apr=0.03,nu=60,step =6):
    space_start = int(ns + step - ns %step)
    space = list(range(space_start,nu+1,step))
    print(space)
    low = 0
    up = len(space)
    while up - low > 1:
        cur_index = int((low+up) / 2)
        cur_n = space[cur_index]
        cal_apr = calculate_rate(pv,cur_n,p)
        if cal_apr == t_apr:
            return cal_apr,cur_n
        elif cal_apr < t_apr:
            low = cur_index
        elif cal_apr > t_apr:
            up = cur_index
    print(up)
    up_cal = calculate_rate(pv,space[up],p)
    low_cal = calculate_rate(pv,space[low],p)
    if abs(up_cal - t_apr) <= abs(low_cal - t_apr):
        return up_cal,space[up]
    else:
        return low_cal,space[low]

In [165]:
print(pv)
print(p)
print(ns)

1293502.5
35698.5
37


In [166]:
estimate_terms(pv,p,ns)

[42, 48, 54, 60]
4


IndexError: list index out of range

## Rough Estimate

In [157]:

pvs = X_cash.AMT_CREDIT.values
payments = X_cash.AMT_ANNUITY.values
low_lengths = X_cash.lowLimitLength.values
rates = []
terms = []
for index in tqdm(range(len(X_cash))):
    pv = pvs[index]
    p = payments[index]
    ns = low_lengths[index]
    r,l = estimate_terms(pv,p,ns)
    rates.append(r)
    terms.append(l)
X_cash['estimate_apr'] = rates
X_cash['estimate_terms'] = terms

  0%|          | 0/326501 [00:00<?, ?it/s]


IndexError: list index out of range

In [107]:
pv = 979992.0
p = 27076.5
l =48
calculate_rate(pv,l,p)

0.01217

In [100]:
pv = 161730.0
p = 13833.0
ns =24
estimate_terms(pv,p,ns)

(0.06783, 24)

In [137]:
X_cash.lowLimitLength.max()

46